# Documentação Útil

In [ ]:
#Requisitar no servidor para acessar a página todo o 'código fonte'
from urllib.request import urlopen

#Transforma a resposta da página para que possamos acessar de fato de forma fácil
from bs4 import BeautifulSoup

#O Parser é o que possibilita pegar o arquivo html e entender(interpretar) o que é uma tag, classe etc
soup = BeautifulSoup(html, 'html.parser')

#Strip: Remove os espaços em branco do início e do fim de cada bloco de texto retornado
soup.get_text(separator=' || ', strip=True)

# Importação Bibliotecas

In [ ]:
# Importando bibliotecas
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

# Acessando o site e a página

In [ ]:
# Declarando variável cards
cards = []
# Obtendo o HTML e o Total de páginas
response = urlopen('https://alura-site-scraping.herokuapp.com/index.php')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')
pages = int(soup.find('span',class_='info-pages').get_text().split()[-1])

# Corpo do Código

In [ ]:
# Iterando por todas as páginas do site
for i in range(pages):
    # Obtendo o HTML
    response = urlopen('https://alura-site-scraping.herokuapp.com/index.php?page='+ str(i + 1))
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')   
    # Obtendo as TAGs de interesse
    anuncios = soup.find('div',{'id':'container-cards'}).findAll('div',class_='card')
    # Coletando as informações dos cards
    for anuncio in anuncios:
        card = {}

        #Valor
        card['value'] = anuncio.find('p',{'class':'txt-value'}).getText()

        #Informações
        infos = anuncio.find('div',{'class':'body-card'}).findAll('p')
        for info in infos:
            card[info.get('class')[0].split('-')[-1]] = info.get_text()
            
        #Acessórios
        items = anuncio.find('div',{'class':'body-card'}).find('ul').findAll('li')
        items.pop()
        acessorios = []
        for item in items:
            acessorios.append(item.getText().replace('► ', ''))
        card['items'] = acessorios

        # Adicionando resultado à lista cards
        cards.append(card)

        #Imagens
        image = anuncio.find('div',{'class':'image-card'}).img
        urlretrieve(image.get('src'),'./output/img/'+ image.get('src').split('/')[-1])





# Exportando para CSV

In [ ]:
# Criando um DataFrame com os resultados
dataset = pd.DataFrame(cards)
dataset.to_csv('./output/data/dataset.csv', sep=';', index = False, encoding = 'utf-8-sig')
dataset